In [1]:
                                          
from pathlib import Path
import os
try :
    PROJECT_DIR =Path (globals ().get ('_dh',[Path .cwd ()])[0 ]).resolve ()
except Exception :
    PROJECT_DIR =Path .cwd ().resolve ()
DATA_DIR =PROJECT_DIR
os .makedirs (DATA_DIR ,exist_ok =True )
print (f"DATA_DIR set to: {DATA_DIR}")


DATA_DIR set to: C:\Users\DELL\OneDrive\Desktop\Final


In [2]:
                                       
TOP_Ks =[5 ,10 ]
SEED =42


In [3]:
                                          
import numpy as np ,pandas as pd ,random ,re
from math import log2
from collections import defaultdict
rng =np .random .default_rng (SEED )
random .seed (SEED )


In [4]:
                            
def clean (text ):
    return re .sub (r"[^\w\s]","",str (text ).lower ()).strip ()
def goal_match_mask (trainer_specialities ,user_goal ):
    tokens =[clean (x )for x in trainer_specialities .split (",")]
    g =clean (user_goal )
    return any (tok in g or g in tok for tok in tokens )
def precision_at_k (y_true ,ranked ,k ):
    hits =sum (1 for i in ranked [:k ]if y_true .get (i ,0 )>0 )
    return hits /k
def recall_at_k (y_true ,ranked ,k ):
    if len (y_true )==0 :
        return np .nan
    hits =sum (1 for i in ranked [:k ]if y_true .get (i ,0 )>0 )
    return hits /len (y_true )
def ndcg_at_k (y_true ,ranked ,k ):
    if not y_true :
        return np .nan
    rel =[y_true .get (i ,0 )for i in ranked [:k ]]
    ideal =sorted (y_true .values (),reverse =True )[:k ]
    dcg =sum (r /log2 (idx +2 )for idx ,r in enumerate (rel ))
    idcg =sum (r /log2 (idx +2 )for idx ,r in enumerate (ideal ))
    return dcg /idcg if idcg >0 else np .nan
def average_precision (y_true ,ranked ):
    """AP for one query, binary relevance."""
    hits ,precisions =0 ,[]
    for idx ,item in enumerate (ranked ,start =1 ):
        if y_true .get (item ,0 )>0 :
            hits +=1
            precisions .append (hits /idx )
    return np .mean (precisions )if precisions else 0.0
def catalog_coverage (all_lists ,k ,catalogue ):
    seen ={i for lst in all_lists for i in lst [:k ]}
    return len (seen )/len (catalogue )
STOP ={"and","&","the","of","for","to","goals","wellness"}
SYN_MAP ={
"aesthetic":["bodybuilding","physique","hypertrophy"],
"physique":["bodybuilding","physique","hypertrophy"],
"senior":["senior","aging","older","balance","functional"],
"aging":["senior","aging","older","balance","functional"],
"wellness":["senior","holistic","balance"],
"stress":["yoga","meditation","breath","mindfulness"],
"mindfulness":["yoga","meditation","breath","mindfulness"],
}
def bag_of_words (text ):
    """return a set of lower-case stemmed tokens minus stop-words"""
    words =re .sub (r"[^\w\s]"," ",str (text ).lower ()).split ()
    return {w .rstrip ("s")for w in words if w not in STOP }


In [5]:
                               
users =pd .read_csv (str (DATA_DIR /'user.csv'))
trainers =pd .read_csv (str (DATA_DIR /'PT_data.cleaned.csv'))
events =pd .read_csv (str (DATA_DIR /'engagement_data.csv'),
parse_dates =["timestamp"])
users ["health_goal_clean"]=users ["health_goal"].apply (clean )
trainers ["specialities_clean"]=trainers ["specialities"].apply (clean )


In [6]:
                            
events ["week"]=events ["timestamp"].dt .to_period ("W").apply (lambda p :p .start_time )
_ev =events .sort_values (["user_id","timestamp"]).copy ()
last_ts =_ev .groupby ("user_id")["timestamp"].max ()
cutoff =(last_ts -pd .Timedelta (days =7 )).rename ("cutoff")
_ev =_ev .merge (cutoff .reset_index (),on ="user_id",how ="left")
_ev ["_in_last7"]=_ev ["timestamp"]>=_ev ["cutoff"]
win_counts =_ev .groupby ("user_id")["_in_last7"].sum ()
ok_users =win_counts [win_counts >=2 ].index
fb_users =win_counts [win_counts <2 ].index
order_cols =["user_id","timestamp"]
if "interaction_id"in _ev .columns :
    order_cols .append ("interaction_id")
last_idx_fb =(
_ev .loc [_ev ["user_id"].isin (fb_users )]
.sort_values (order_cols )
.groupby ("user_id",as_index =False ).tail (1 ).index
)
test_mask =(_ev ["_in_last7"]&_ev ["user_id"].isin (ok_users ))|(_ev .index .isin (last_idx_fb ))
train_mask =~test_mask
train =_ev .loc [train_mask ].drop (columns =["cutoff","_in_last7"]).copy ()
test =_ev .loc [test_mask ].drop (columns =["cutoff","_in_last7"]).copy ()
print (f"Train rows: {len(train):,}   Test rows: {len(test):,}")


Train rows: 2,959,786   Test rows: 160,324


In [7]:
                                          
from difflib import SequenceMatcher
import re
def tokens (text ):
    return set (re .sub (r"[^\w\s]"," ",str (text ).lower ()).split ())
def fuzzy_match (goal ,specs ,thresh =0.75 ):
    for s in specs :
        if SequenceMatcher (None ,goal .lower (),s .lower ()).ratio ()>=thresh :
            return True
    return False
def matched_trainers (user_row ):
    goal_raw =user_row ["health_goal"]
    goal_tok =tokens (goal_raw )
    mask =trainers ["specialities"].apply (
    lambda spec :bool (tokens (spec )&goal_tok )
    )
    cand =trainers .loc [mask ,"trainer_id"].tolist ()
    if cand :
        return cand
    syns =set ()
    for t in goal_tok :
        syns |=set (SYN_MAP .get (t ,[]))
    if syns :
        mask =trainers ["specialities"].apply (
        lambda spec :bool (tokens (spec )&syns )
        )
        cand =trainers .loc [mask ,"trainer_id"].tolist ()
        if cand :
            return cand
    mask =trainers ["specialities"].apply (
    lambda spec :fuzzy_match (goal_raw ,spec .split (","))
    )
    cand =trainers .loc [mask ,"trainer_id"].tolist ()
    if cand :
        return cand
    return fallback_order
pop_global_df =(
train
.groupby ('trainer_id',as_index =False )
.agg (count =('trainer_id','size'),last_ts =('timestamp','max'))
.sort_values (['count','last_ts'],ascending =[False ,False ])
)
global_rank ={tid :i for i ,tid in enumerate (pop_global_df ['trainer_id'].tolist ())}
fallback_order =trainers ['trainer_id'].tolist ()
fallback_tail =[t for t in fallback_order if t not in global_rank ]
fallback_rank ={tid :len (global_rank )+i for i ,tid in enumerate (fallback_tail )}


In [8]:
                                       
def random_ranker (user_row ):
    cands =matched_trainers (user_row )
    rng .shuffle (cands )
    return cands
def popularity_ranker (user_row ):
    cands =matched_trainers (user_row )
    if not cands :
        return []
    ranked =sorted (
    cands ,
    key =lambda tid :(global_rank .get (tid ,float ('inf')),
    fallback_rank .get (tid ,float ('inf')))
    )
    return ranked
baselines =[("random",random_ranker ),
("popularity_global",popularity_ranker )]


In [9]:
                            
results =defaultdict (list )
lists_for_cov ={n :[]for n ,_ in baselines }
catalogue =set (trainers .trainer_id )
for (uid ,wk ),grp in test .groupby (["user_id","week"]):
    user_row =users .loc [users .user_id ==uid ].iloc [0 ]
    y_true ={tid :1 for tid in grp .trainer_id .unique ()}
    for name ,ranker in baselines :
        ranked =ranker (user_row )
        if not ranked :
            continue
        lists_for_cov [name ].append (ranked )
        for k in TOP_Ks :
            results ["model"].append (name )
            results ["k"].append (k )
            results ["precision"].append (precision_at_k (y_true ,ranked ,k ))
            results ["recall"].append (recall_at_k (y_true ,ranked ,k ))
            results ["ndcg"].append (ndcg_at_k (y_true ,ranked ,k ))
            results ["map"].append (np .nan )
            results ["ranked_list"].append (ranked )
            results ["top_trainer"].append (str (ranked [0 ]))
            results ["week"].append (wk )
            results ["user_id"].append (str (uid ))
        results ["model"].append (name )
        results ["k"].append ("MAP")
        results ["precision"].append (np .nan )
        results ["recall"].append (np .nan )
        results ["ndcg"].append (np .nan )
        results ["map"].append (average_precision (y_true ,ranked ))
        results ["ranked_list"].append (ranked )
        results ["top_trainer"].append (ranked [0 ])
        results ["week"].append (wk )
        results ["user_id"].append (uid )
df =pd .DataFrame (results )
acc_df =(df [df ["k"].isin (TOP_Ks )]
.groupby (["model","k"])
.mean (numeric_only =True )
.round (4 )
.reset_index ()
.drop (columns ="map"))
map_df =(df [df ["k"]=="MAP"]
.groupby ("model")["map"]
.mean ()
.round (4 )
.reset_index ())
cov_rows =[]
for name in lists_for_cov :
    for k in TOP_Ks :
        cov_rows .append ({"model":name ,
        "k":k ,
        "coverage":round (
        catalog_coverage (lists_for_cov [name ],k ,catalogue ),4 )})
cov_df =pd .DataFrame (cov_rows )


In [10]:
                           
print ("Accuracy (Precision / Recall / NDCG):")
display (acc_df )
print ("\nMean Average Precision (full list):")
display (map_df )
print ("\nCatalog Coverage:")
display (cov_df )


Accuracy (Precision / Recall / NDCG):


,model,k,precision,recall,ndcg
0,popularity_global,5,0.0112,0.0249,0.0191
1,popularity_global,10,0.0103,0.0462,0.0275
2,random,5,0.0063,0.0142,0.0105
3,random,10,0.0063,0.0287,0.0162



Mean Average Precision (full list):


,model,map
0,popularity_global,0.0374
1,random,0.0249



Catalog Coverage:


,model,k,coverage
0,random,5,0.873
1,random,10,0.873
2,popularity_global,5,0.037
3,popularity_global,10,0.070


In [11]:
                                          
import pandas as pd
lists_df =(df [df ["k"]=="MAP"]
.loc [:,["user_id","week","model","ranked_list"]])
top5_df =(lists_df
.assign (top5 =lambda d :d ["ranked_list"].apply (lambda x :x [:5 ]))
.loc [:,["user_id","model","top5"]]
.sort_values ("user_id")
.reset_index (drop =True ))
display (top5_df )


,user_id,model,top5
0,1,random,"[573, 19, 648, 798, 644]"
1,1,popularity_global,"[172, 456, 515, 317, 615]"
2,1,random,"[943, 455, 475, 438, 318]"
3,1,popularity_global,"[172, 456, 515, 317, 615]"
4,2,random,"[613, 971, 207, 70, 720]"
...,...,...,...
141745,39999,popularity_global,"[267, 614, 661, 683, 74]"
141746,40000,popularity_global,"[797, 847, 138, 738, 267]"
141747,40000,random,"[131, 152, 458, 853, 496]"
141748,40000,random,"[570, 612, 156, 665, 222]"


In [12]:
                                  
top5_df .head (10 )


,user_id,model,top5
0,1,random,"[573, 19, 648, 798, 644]"
1,1,popularity_global,"[172, 456, 515, 317, 615]"
2,1,random,"[943, 455, 475, 438, 318]"
3,1,popularity_global,"[172, 456, 515, 317, 615]"
4,2,random,"[613, 971, 207, 70, 720]"
5,2,popularity_global,"[416, 38, 118, 484, 341]"
6,3,random,"[981, 703, 400, 840, 866]"
7,3,popularity_global,"[885, 622, 847, 80, 748]"
8,3,random,"[135, 956, 321, 80, 286]"
9,3,popularity_global,"[885, 622, 847, 80, 748]"


In [13]:
                           
no_cand_users =[]
for uid in users .user_id .unique ():
    user_row =users [users .user_id ==uid ].iloc [0 ]
    if not matched_trainers (user_row ):
        no_cand_users .append (uid )
print (f"{len(no_cand_users)} users have no goal-matching trainers.")
with_cand =set (users .user_id )-set (no_cand_users )
missing =with_cand -set (lists_df .user_id .unique ())
print (f"{len(missing)} users had a candidate list but no held-out week.")


0 users have no goal-matching trainers.
0 users had a candidate list but no held-out week.
